In [ ]:
import pickle
import os
import pandas as pd
import seaborn as sn
import numpy as np
np.random.seed(42)
from sklearn.model_selection import train_test_split as splt
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import EarlyStopping
import seaborn as sns
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
df = pd.read_csv("Data/new_wScore.csv")

In [ ]:
df.sort_values("visit")
df = df.select_dtypes(exclude=['object', 'datetime64'])
df = df.drop(labels = ['SCORE','ana_fis:smoking_recod', 'lab:glucose', 'lab:calculated_ldl', 'lab:total_cholesterol'], axis=1)
df = df.fillna(-1).replace(-1, np.nan).replace("-1", np.nan).replace(-1.0, np.nan)
df = df.dropna(how='any', axis=1)

In [ ]:
df.head()

In [ ]:
d = {i: df.loc[df.subject_id == i, df.columns] for i in range(df.subject_id.iat[-1]+1)}
d = {dx: d[dx] for dx in d if d[dx].shape[0] != 0}
#X = np.array([np.array(values)[:-1,:-1] for key, values in d.items()])
#X = np.array([X[:,:,i]/X[:,:,i].max() for i in range(X.shape[2])])
#print(X.shape)
#X = np.swapaxes(X, 0, 1)
#X = np.swapaxes(X, 1, 2)
#y = np.array([np.array(values)[-1,-1] for key, values in d.items()])
#yy = np.zeros((y.shape[0], 4))
#for i,yyy in enumerate(y):
#    if(yyy < 0.01): # Low
#       yy[i][0] = 1
#    elif(yyy < 0.02): # Medium
#        yy[i][1] = 1
#    elif(yyy < 0.05): # High
#        yy[i][2] = 1
#    else: # Very High (aka Dead)
#        yy[i][3] = 1 

In [ ]:
X1 = []
y1 = []
X2 = []
y2 = []
X3 = []
y3 = []
for key, value in d.items():
    row1 = value.iloc[[0]]
    row2 = value.iloc[[1]]
    row3 = value.iloc[[2]]
    row4 = value.iloc[[3]]
    val = np.zeros(4)
    val[row2.ScoreClass-1] = 1
    y1.append(val)
    X1.append(row1.drop('ScoreClass', axis=1).values[0])
    
    val = np.zeros(4)
    val[row3.ScoreClass-1] = 1
    y1.append(val)
    X1.append(row2.drop('ScoreClass', axis=1).values[0])
    
    val = np.zeros(4)
    val[row4.ScoreClass-1] = 1
    y1.append(val)
    X1.append(row3.drop('ScoreClass', axis=1).values[0])
    
    val = np.zeros(4)
    val[row3.ScoreClass-1] = 1
    y2.append(val)
    X2.append(row1.drop('ScoreClass', axis=1).values[0])
    
    val = np.zeros(4)
    val[row4.ScoreClass-1] = 1
    y2.append(val)
    X2.append(row2.drop('ScoreClass', axis=1).values[0])
    
    val = np.zeros(4)
    val[row4.ScoreClass-1] = 1
    y3.append(val)
    X3.append(row1.drop('ScoreClass', axis=1).values[0])

In [ ]:
X1 = np.array(X1)
X2 = np.array(X2)
X3 = np.array(X3)
y1 = np.array(y1)
y2 = np.array(y2)
y3 = np.array(y3)

In [ ]:
y1[0]

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# First Model

In [ ]:
model1 = Sequential()
model1.add(Dense(32, input_shape=(X1.shape[1], ), activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(16, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(8, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))
model1.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_tr, X_ts, y_tr, y_ts = splt(X1, y1, test_size=0.25, random_state=42)
es = EarlyStopping(patience = 20, restore_best_weights = True)
history = model1.fit(X_tr, y_tr, validation_data=(X_ts, y_ts), epochs = 340)#, callbacks = [es])

In [ ]:
plot_history(history)

In [ ]:
y_ts_pred = model1.predict(X_ts)
cmatrix = np.zeros((4,4))
for i,y_our in enumerate(y_ts_pred):
    cmatrix[np.argmax(y_our)][np.argmax(y_ts[i])] += 1

In [ ]:
df_cm = pd.DataFrame(cmatrix, index = ["No", "Low", "Med", "High"], columns = ["No", "Low", "Med", "High"])
plt.figure(figsize = (10,7))
snheatmap = sn.heatmap(df_cm, annot=True,  fmt='g', cmap="Blues")

# Second Model

In [ ]:
model2 = Sequential()
model2.add(Dense(32, input_shape=(X2.shape[1], ), activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(16, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(8, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(4, activation='softmax'))
model2.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_tr, X_ts, y_tr, y_ts = splt(X2, y2, test_size=0.25, random_state=42)
es = EarlyStopping(patience = 20, restore_best_weights = True)
history = model2.fit(X_tr, y_tr, validation_data=(X_ts, y_ts), epochs = 340)#, callbacks = [es])

In [ ]:
y_ts_pred = model2.predict(X_ts)
cmatrix = np.zeros((4,4))
for i,y_our in enumerate(y_ts_pred):
    cmatrix[np.argmax(y_our)][np.argmax(y_ts[i])] += 1

In [ ]:
df_cm = pd.DataFrame(cmatrix, index = ["No", "Low", "Med", "High"], columns = ["No", "Low", "Med", "High"])
plt.figure(figsize = (10,7))
snheatmap = sn.heatmap(df_cm, annot=True,  fmt='g', cmap="Blues")

# Third Model

In [ ]:
model3 = Sequential()
model3.add(Dense(32, input_shape=(X3.shape[1], ), activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(16, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(8, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(4, activation='softmax'))
model3.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_tr, X_ts, y_tr, y_ts = splt(X3, y3, test_size=0.25, random_state=42)
es = EarlyStopping(patience = 20, restore_best_weights = True)
history = model3.fit(X_tr, y_tr, validation_data=(X_ts, y_ts), epochs = 340)#, callbacks = [es])

In [ ]:
y_ts_pred = model3.predict(X_ts)
cmatrix = np.zeros((4,4))
for i,y_our in enumerate(y_ts_pred):
    cmatrix[np.argmax(y_our)][np.argmax(y_ts[i])] += 1

In [ ]:
df_cm = pd.DataFrame(cmatrix, index = ["No", "Low", "Med", "High"], columns = ["No", "Low", "Med", "High"])
plt.figure(figsize = (10,7))
snheatmap = sn.heatmap(df_cm, annot=True,  fmt='g', cmap="Blues")